# 광주 공원 추천 시스템

In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

In [2]:
# 데이터 불러오기

data = pd.read_csv('data/park_list1.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   공원명       26 non-null     object 
 1   키워드       20 non-null     object 
 2   키워드 (영문)  26 non-null     object 
 3   평점        26 non-null     float64
 4   평점투표수     26 non-null     int64  
 5   내부시설      21 non-null     object 
 6   비고        1 non-null      object 
dtypes: float64(1), int64(1), object(5)
memory usage: 1.5+ KB


In [3]:
# 키워드를 입력 받기
# 입력 예시 : strolling, workout

print('산책, 운동, 도심, 걷기, 가족, 문화, 데이트, 벚꽃')
keyword = (input('키워드를 선택하세요(중복 선택 가능, 영어로 입력) '))

산책, 운동, 도심, 걷기, 가족, 문화, 데이트, 벚꽃
키워드를 선택하세요(중복 선택 가능, 영어로 입력) strolling, workout


In [4]:
# 머무를 시간을 입력 받기
# 입력 예시 : 30분 이내

print('30분 이내, 1시간 이내, 2시간 이내, 3시간 이내')
time = (input('머무를 시간을 선택하세요'))

30분 이내, 1시간 이내, 2시간 이내, 3시간 이내
머무를 시간을 선택하세요30분 이내


In [5]:
# 데이터 전처리

keyword = keyword.replace(',', '')
keyword = keyword.split()

data['키워드 (영문)'] = data['키워드 (영문)'].str.replace(',', '')

In [6]:
# 입력받은 키워드를 많이 포함하고 있는 키워드를 가진 공원 추출

data['중복'] = ''

for i in range(len(data)):
    data['중복'][i] = list(set(keyword).intersection(list(map(str, data['키워드 (영문)'][i].split()))))
    
data['카운트'] = 0

for i in range(len(data)):
    data['카운트'][i] = len(data['중복'][i])
    
print("키워드와 관련 있는 공원 : ", data.sort_values('카운트', ascending=False).iloc[0,0])

키워드와 관련 있는 공원 :  5.18기념


In [7]:
# CountVectorizer를 이용해 피처 벡터 행렬로 변환
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
keyword_mat = vect.fit_transform(data['키워드 (영문)'])

In [8]:
# 코사인 유사도 계산계산
from sklearn.metrics.pairwise import cosine_similarity

keyword_sim = cosine_similarity(keyword_mat, keyword_mat)

In [9]:
# 유사도가 높은 순으로 정리된 genre_sim 객체의 비교 행 위치 인덱스 값
keyword_sim_sorted_ind = keyword_sim.argsort()[:, ::-1]

In [10]:
# 키워드 유사도에 따라 공원을 추천하고 평점 순으로 나열

def park_recommend(data, sorted_ind, title_name, top_n=10):
    title_park = data[data['공원명'] == title_name]
    title_index = title_park.index.values
    sim_indexs = sorted_ind[title_index, :top_n]
    
    sim_indexs = sim_indexs.reshape(-1)
    
    return data.iloc[sim_indexs]

In [11]:
# 공원 면적 데이터 불러오기

info = pd.read_csv('data/park_info.csv', encoding='cp949')
info = info[['공원명','공원면적']]
info.head()

data = pd.merge(data, info, how='left')

In [12]:
if time == '30분 이내':
    time = 0.5
elif time == '1시간 이내':
    time = 1
elif time == '2시간 이내':
    time = 2
elif time == '3시간 이내':
    time = 3

In [13]:
data['공원둘레'] = (np.sqrt(data['공원면적']) * 4)

In [14]:
# 공원 걷는 속도를 2km/h 라고 가정

walk = time * 2 * 1000

data['walk'] = data['공원둘레'] > walk

# 실행 결과

In [15]:
# 머무를 시간이 부족한 공원은 추천 목록에서 제외

df = park_recommend(data, keyword_sim_sorted_ind, data.sort_values('카운트', ascending=False).iloc[0,0])
df = df[df['walk'] == False]
df = df.iloc[1:,:]

print("추천 1순위 : ", df.iloc[0,0])
print("추천 2순위 : ", df.iloc[1,0])
print("추천 3순위 : ", df.iloc[2,0])

추천 1순위 :  유안
추천 2순위 :  우산
추천 3순위 :  쌍학
